## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-03-23-43-05 +0000,nyt,Venezuela Live Updates: Maduro Arrives in N.Y....,https://www.nytimes.com/live/2026/01/03/world/...
1,2026-01-03-23-42-43 +0000,bbc,"Spies, drones and blowtorches: How the US capt...",https://www.bbc.com/news/articles/cdred61epg4o...
2,2026-01-03-23-41-09 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/01/03/world/...
3,2026-01-03-23-40-13 +0000,nyt,The Year in Neanderthal Discoveries,https://www.nytimes.com/2026/01/03/science/arc...
4,2026-01-03-23-36-08 +0000,nyt,A Pardon and a Prosecution in New York Show Tr...,https://www.nytimes.com/2026/01/03/world/ameri...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2367/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,maduro,50
0,venezuela,48
5,trump,38
26,capture,19
19,new,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
0,2026-01-03-23-43-05 +0000,nyt,Venezuela Live Updates: Maduro Arrives in N.Y....,https://www.nytimes.com/live/2026/01/03/world/...,207
36,2026-01-03-22-19-02 +0000,nypost,"Nicolás Maduro, Cilia Flores land in New York ...",https://nypost.com/2026/01/03/us-news/nicolas-...,166
78,2026-01-03-18-17-46 +0000,nypost,Trump brushes off concerns about legality of U...,https://nypost.com/2026/01/03/world-news/trump...,163
75,2026-01-03-18-42-03 +0000,latimes,Trump says U.S. will 'run' Venezuela after cap...,https://www.latimes.com/politics/story/2026-01...,161
48,2026-01-03-21-22-04 +0000,nyt,Venezuela’s Interim Leader Defies Trump and Ca...,https://www.nytimes.com/2026/01/03/world/ameri...,156


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
0,207,2026-01-03-23-43-05 +0000,nyt,Venezuela Live Updates: Maduro Arrives in N.Y....,https://www.nytimes.com/live/2026/01/03/world/...
36,68,2026-01-03-22-19-02 +0000,nypost,"Nicolás Maduro, Cilia Flores land in New York ...",https://nypost.com/2026/01/03/us-news/nicolas-...
164,48,2026-01-03-03-00-00 +0000,wsj,The massive welfare-fraud scandal in Minnesota...,https://www.wsj.com/politics/policy/minnesota-...
49,37,2026-01-03-21-20-02 +0000,bbc,Swiss open criminal case against managers of s...,https://www.bbc.com/news/articles/cjdjl303zxlo...
83,31,2026-01-03-17-50-54 +0000,nypost,"Venezuelan dictator appears blindfolded, handc...",https://nypost.com/2026/01/03/world-news/venez...
152,30,2026-01-03-05-09-32 +0000,nypost,DOJ official warns Seattle mayor to leave jour...,https://nypost.com/2026/01/03/us-news/doj-offi...
73,30,2026-01-03-19-04-15 +0000,nypost,Inside Operation Absolute Resolve: How US forc...,https://nypost.com/2026/01/03/world-news/how-u...
62,19,2026-01-03-20-15-35 +0000,nypost,World leaders gripe over US arrest of tyrant N...,https://nypost.com/2026/01/03/world-news/world...
63,19,2026-01-03-20-14-35 +0000,bbc,Starmer won't be drawn on whether US strikes o...,https://www.bbc.com/news/articles/cy4qgvwxp08o...
27,18,2026-01-03-22-50-03 +0000,nypost,Mamdani schools chancellor pick an ‘idiot’ and...,https://nypost.com/2026/01/03/us-news/mamdani-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
